In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import *
import pandas as pd

In [3]:
import ast

def get_names(element):
    return [info['name'] for info in element]

def join_names(element):
    return ' '.join(element)

filepath='D:/downloads/3405_6663_bundle_archive/'
filename='tmdb_5000_movies.csv'
data=pd.read_csv(filepath+filename)
data=data[['id','genres','vote_average','vote_count','popularity','title','keywords','overview']]
tmp_m=data['vote_count'].quantile(0.90)
data=data[data['vote_count']>=tmp_m]
C = data['vote_average'].mean()
M = tmp_m
def weighted_rating(x,m=M,c=C):
    v=x['vote_count']
    R=x['vote_average']
    
    return (v/(v+m)*R) + (m/(m+v)*C)
data['score'] = data.apply(weighted_rating,axis=1)
data['genres']=data['genres'].apply(ast.literal_eval)
data['keywords']=data['keywords'].apply(ast.literal_eval)

data['genres']=data['genres'].apply(get_names).apply(join_names)
data['keywords']=data['keywords'].apply(get_names).apply(join_names)
data[['genres','keywords']].head(2)

,genres,keywords
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...


In [4]:
count_vector = CountVectorizer(ngram_range=(1,3))
c_vector_genres = count_vector.fit_transform(data['genres'])
print(c_vector_genres.toarray())
print(c_vector_genres.shape)

[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(481, 364)


In [9]:
genre_c_sim=cosine_similarity(c_vector_genres,c_vector_genres).argsort()[:,::-1]

In [ ]:
def get_recommend_movie_list(db,title,top=30):
    target_movie_index = db[db['title']==title].index.values
    ret_index= genre_c_sim[target_movie_index,:30].reshape(-1)
    print(ret_index)
    for i in ret_index:
        print(db)
    return

get_recommend_movie_list(data,'The Dark Knight Rises')

__아이템 기반 협업 필터링 : 사용자 행동 피턴을 기반으로 추천__

In [37]:
filepath='D:/downloads/3405_6663_bundle_archive/'
filename='ratings_small.csv'
data=pd.read_csv(filepath+filename)
data.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [38]:
data=data.pivot_table('rating',index='userId',columns='movieId')
data.head(2)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
data.shape  # 671 명 유저 , 9066 개 영화 

(671, 9066)

In [31]:
filepath='D:/downloads/3405_6663_bundle_archive/'
filename='tmdb_5000_movies.csv'
movies=pd.read_csv(filepath+filename)
movies.rename(columns={'id':'movieId'},inplace=True)
movies.head(2)

,budget,genres,homepage,movieId,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [36]:
movies.columns

Index(['budget', 'genres', 'homepage', 'movieId', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count'],
      dtype='object')

In [42]:
filepath='D:/downloads/3405_6663_bundle_archive/'
filename='ratings_small.csv'
ratings=pd.read_csv(filepath+filename)

ratings_movies=pd.merge(ratings,movies,on='movieId')
ratings_movies.head(3)

,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,2105,4.0,1260759139,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
1,4,2105,4.0,949896114,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
2,15,2105,4.0,1052896867,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296


In [41]:
ratings_movies.columns

Index(['userId', 'movieId', 'rating', 'timestamp', 'budget', 'genres',
       'homepage', 'keywords', 'original_language', 'original_title',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [43]:
ratings_movies[ratings_movies['userId']==1]

,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,2105,4.0,1260759139,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
47,1,2294,2.0,1260759108,22000000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,"[{""id"": 886, ""name"": ""film making""}, {""id"": 16...",en,Jay and Silent Bob Strike Back,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2001-08-22,33788161,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Hollywood had it coming,Jay and Silent Bob Strike Back,6.4,480


__예산 대비 인기도 = ?__  
* x:예산 , y:인기도  

In [46]:
ratings_movies.shape

(18571, 23)

In [50]:
data_=ratings_movies.pivot_table('rating',index='userId',columns='title').fillna(0) # 사용자별 각 영화 평점
data_.shape # 670,856
data_.head()

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
data_.T

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Blocks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"You, Me and Dupree",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Young Frankenstein,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Zodiac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
movie_sim = cosine_similarity(data_.T,data_.T)
movie_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.01412134],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.16380117,
        0.10537865],
       [0.        , 0.        , 0.        , ..., 0.16380117, 1.        ,
        0.0441038 ],
       [0.01412134, 0.        , 0.        , ..., 0.10537865, 0.0441038 ,
        1.        ]])

In [60]:
movie_sim_df=pd.DataFrame(movie_sim,index=data_.T.index,columns=data_.T.index)
movie_sim_df.head(3)

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,1.0,0.0,0.0,0.182153,0.0,0.022069,0.085323,0.0,0.0,0.10349,...,0.059856,0.0,0.161801,0.088076,0.0,0.0,0.097588,0.0,0.0,0.014121
12 Angry Men,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1408,0.0,0.0,1.0,0.447214,0.0,0.173381,0.028245,0.0,0.0,0.00000,...,0.146955,0.0,0.148968,0.140265,0.0,0.0,0.191675,0.0,0.0,0.000000


In [63]:
movie_sim_df['X-Men Origins: Wolverine'].sort_values(ascending=False)

title
X-Men Origins: Wolverine    1.000000
Romeo Must Die              0.649625
The Wedding Planner         0.631669
Dogtown and Z-Boys          0.501189
An Unfinished Life          0.485643
                              ...   
Deep Impact                 0.000000
The Evil Dead               0.000000
Dead Man Walking            0.000000
Lion of the Desert          0.000000
Mr. & Mrs. Smith            0.000000
Name: X-Men Origins: Wolverine, Length: 856, dtype: float64